# WordNet Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

wordnet_path = '/data/image-net/ILSVRC2015/devkit/data/map_clsloc.txt'
wordnet_dic = {}

with open(wordnet_path) as file:
    for line in file:
        wordnet_dic[line.split()[0]] = line.split()
        
le = LabelEncoder()

le.fit(list(wordnet_dic.keys()))

print(le.classes_)

In [ ]:
print(le.transform(['n02119789', 'n02100735', 'n02110185', 'n02096294']))

print(le.inverse_transform([278, 212, 250, 193]))

In [ ]:
def wordnet_label_encoder(wordnet_label):
    label = le.transform([wordnet_label])[0]
    return label

# ImageNet Dataset

In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

print('executing_eagerly = ' + str(tf.executing_eagerly()))

# Reads an image from a file, decodes it into a dense tensor
def load_image(filename):
    wordnet_label = tf.strings.split(filename, os.sep)[-2]
    label = tf.numpy_function(wordnet_label_encoder, [wordnet_label], [tf.int64])
    label = tf.squeeze(label)
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, label


def show(image, label):
    plt.figure()
    plt.imshow(image)
    plt.title(label.numpy())

In [ ]:
imagenet_train_dir = '/data/image-net/ILSVRC2012_img_train/'
imagenet_files = tf.data.Dataset.list_files(imagenet_train_dir + '*/*')
imagenet_train_ds = imagenet_files.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
for image, label in imagenet_train_ds.take(5):
    show(image, label)

# ImageNet Pipeline

In [ ]:
# image transform

image_size = [224, 224]

def image_transform(image, label):
    # resize
    image = tf.image.resize(image, image_size)
    # normalize
    return image, label

imagenet_train_ds = imagenet_train_ds.map(image_transform, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
for image, label in imagenet_train_ds.take(5):
    show(image, label)

In [ ]:
# batch
batch_size = 32

imagenet_train_ds = imagenet_train_ds.batch(batch_size)

In [ ]:
# prefetch
imagenet_train_ds = imagenet_train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
# cache 
imagenet_train_ds = imagenet_train_ds.cache()

# ResNet34V1

In [ ]:
from ResNetV1Block import identity_block
from ResNetV1Block import identity_block_downsampling


def ResNet34V1(
        input_shape=(224, 224, 3),
        classes=1000,
        classifier_activation='softmax'):
    
    img_input = keras.layers.Input(shape=input_shape, name='input')

    # stage1
    x = keras.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', name='stage1_conv')(img_input)
    x = keras.layers.BatchNormalization(axis=-1, name='stage1_bn')(x)
    x = keras.layers.ReLU(name='stage1_relu')(x)
    
    # stage2
    x = keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='stage2_pool')(x)
    x = identity_block(x, filters=64, stage='stage2', block='a')
    x = identity_block(x, filters=64, stage='stage2', block='b')
    x = identity_block(x, filters=64, stage='stage2', block='c')

    # stage3
    x = identity_block_downsampling(x, filters=128, stage='stage3', block='a')
    x = identity_block(x, filters=128, stage='stage3', block='b')
    x = identity_block(x, filters=128, stage='stage3', block='c')
    x = identity_block(x, filters=128, stage='stage3', block='d')
    
    # stage4
    x = identity_block_downsampling(x, filters=256, stage='stage4', block='a')
    x = identity_block(x, filters=256, stage='stage4', block='b')
    x = identity_block(x, filters=256, stage='stage4', block='c')
    x = identity_block(x, filters=256, stage='stage4', block='d')
    x = identity_block(x, filters=256, stage='stage4', block='e')
    x = identity_block(x, filters=256, stage='stage4', block='f')
    
    # stage5
    x = identity_block_downsampling(x, filters=512, stage='stage5', block='a')
    x = identity_block(x, filters=512, stage='stage5', block='b')
    x = identity_block(x, filters=512, stage='stage5', block='c')
    
    # classifier
    x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = keras.layers.Dense(units=classes, activation=classifier_activation, name='predictions')(x)

    # Create model.
    inputs = img_input
    model = keras.Model(inputs=inputs, outputs=x, name='resnet34_v1')

    return model

In [ ]:
model = ResNet34V1()

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "resnet34_v1.png", show_shapes=True)

# Train Model

In [ ]:
# optimizers - losses - metrics
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True, name='SGD')
loss_fn = keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        probs = model(x_batch, training=True)
        loss_value = loss_fn(y, probs)
        loss_value += sum(model.losses)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [ ]:
@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [ ]:
import time

epochs = 1

for epoch in range(epochs):
    
    print("\nStart of epoch %d" % (epoch,))
    
    start_time = time.time()
    
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    
        loss_value = train_step(x_batch_train, y_batch_train)
        
        if step % 200 == 0:
            print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss_value)))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    if val_dataset is not None:
        for x_batch_val, y_batch_val in val_dataset:
            test_step(x_batch_val, y_batch_val)
        val_acc = val_acc_metric.result()
        val_acc_metric.reset_states()
        print("Validation acc: %.4f" % (float(val_acc),))
        
    print("Time taken: %.2fs" % (time.time() - start_time))

# Training checkpoints

In [ ]:
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    # Path where to save the model
    # The two parameters below mean that we will overwrite
    # the current checkpoint if and only if
    # the `val_loss` score has improved.
    # The saved model name will include the current epoch.
    filepath="resnet34_v1_{epoch}",
    save_best_only=True,  # Only save a model if `val_loss` has improved.
    monitor="val_loss",
    verbose=1,
)

In [ ]:
# callbacks
early_stopping = keras.callbacks.EarlyStopping(
    # Stop training when `val_loss` is no longer improving
    monitor="val_loss",
    # "no longer improving" being defined as "no better than 1e-2 less"
    min_delta=1e-2,
    # "no longer improving" being further defined as "for at least 2 epochs"
    patience=2,
    verbose=1,
)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

learning_rate_schedules = tf.keras.callbacks.LearningRateScheduler(scheduler)

tensor_board = keras.callbacks.TensorBoard(
    log_dir="tensor_board_logs",
    histogram_freq=0,  # How often to log histogram visualizations
    embeddings_freq=0,  # How often to log embedding visualizations
    update_freq="epoch",
)  # How often to write logs (default: once per epoch)

callbacks = [
    early_stopping,
    checkpoint,
    learning_rate_schedules,
    tensor_board
]